# scrape Honda vehicles
* webpage: https://www.honda.co.th/models

In [1]:
import sys
import re
import io
import requests

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from PIL import Image # pillow package (Python Image Library)

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [3]:
print(f'pandas version = {pd.__version__}')
print(pd.Timestamp.now())

pandas version = 1.2.3
2021-04-07 14:41:23.803652


In [4]:
from IPython.display import HTML
pd.set_option('display.max_colwidth', 0)

In [6]:
url='https://www.honda.co.th/models'
with requests.session() as s:
    r = s.get(url)
    r.text[:100]

In [7]:
s = BeautifulSoup(r.text, 'lxml')
type(s)

bs4.BeautifulSoup

In [8]:
el=s.select('div.model_overview_list')

print(len(el))
print(el[1].prettify())

13
<div class="model_overview_list" data-aos="fade-up" data-aos-duration="900">
 <a class="model_overview_link gtm-model-cityehev" href="cityehev" target="_blank">
  <div class="tag_new">
   ใหม่
  </div>
  <div class="model_overview_list_logo">
   <img alt="ฮอนด้า City e:HEV" src="https://www.honda.co.th/uploads/car_models/1cb300a181a28cfd97704f57892beada81.png"/>
  </div>
  <div class="model_overview_list_img">
   <img alt="ฮอนด้า City e:HEV" src="https://www.honda.co.th/uploads/car_models/adefeba31088626dfb607d41ea6229cd50.png"/>
  </div>
  <div class="model_overview_list_btn">
   Honda City e:HEV
   <span class="arrow_next">
   </span>
  </div>
 </a>
 <!-- model_overview_link -->
 <div class="model_overview_list_detail">
  <div class="model_overview_list_txt">
   สัมผัสขุมพลังความแรง ประหยัดน้ำมันดีเยี่ยม พร้อมเทคโนโลยีความปลอดภัยอัจฉริยะ ให้คุณพุ่งทะยานสู่อนาคตได้อย่างมั่นใจ
  </div>
  <div class="model_overview_list_txt_price">
   เริ่มต้น 839,000 บาท
  </div>
 </div>
</div>



In [9]:
e=el[1]

In [10]:
print(e.prettify())

<div class="model_overview_list" data-aos="fade-up" data-aos-duration="900">
 <a class="model_overview_link gtm-model-cityehev" href="cityehev" target="_blank">
  <div class="tag_new">
   ใหม่
  </div>
  <div class="model_overview_list_logo">
   <img alt="ฮอนด้า City e:HEV" src="https://www.honda.co.th/uploads/car_models/1cb300a181a28cfd97704f57892beada81.png"/>
  </div>
  <div class="model_overview_list_img">
   <img alt="ฮอนด้า City e:HEV" src="https://www.honda.co.th/uploads/car_models/adefeba31088626dfb607d41ea6229cd50.png"/>
  </div>
  <div class="model_overview_list_btn">
   Honda City e:HEV
   <span class="arrow_next">
   </span>
  </div>
 </a>
 <!-- model_overview_link -->
 <div class="model_overview_list_detail">
  <div class="model_overview_list_txt">
   สัมผัสขุมพลังความแรง ประหยัดน้ำมันดีเยี่ยม พร้อมเทคโนโลยีความปลอดภัยอัจฉริยะ ให้คุณพุ่งทะยานสู่อนาคตได้อย่างมั่นใจ
  </div>
  <div class="model_overview_list_txt_price">
   เริ่มต้น 839,000 บาท
  </div>
 </div>
</div>



In [11]:
img_url=e.select_one('div.model_overview_list_img>img')['src']
img_url

'https://www.honda.co.th/uploads/car_models/adefeba31088626dfb607d41ea6229cd50.png'

In [12]:
model=e.select_one('div.model_overview_list_btn').text.strip()
model

'Honda City e:HEV'

In [14]:
price_thb=pd.to_numeric(e.select_one('div.model_overview_list_txt_price').text.strip().replace('เริ่มต้น ','').replace(' บาท','').replace(',',''))
price_thb

839000

In [15]:
overview=e.select_one('div.model_overview_list_txt').text.strip().replace('\r\n','')
overview

'สัมผัสขุมพลังความแรง ประหยัดน้ำมันดีเยี่ยม พร้อมเทคโนโลยีความปลอดภัยอัจฉริยะ ให้คุณพุ่งทะยานสู่อนาคตได้อย่างมั่นใจ'

In [16]:
records=[]
for e in el[:]:
    img_url=e.select_one('div.model_overview_list_img>img')['src']
    model=e.select_one('div.model_overview_list_btn').text.strip()
    price_thb=e.select_one('div.model_overview_list_txt_price').text.strip()
    price_thb=int(re.sub('\D', '', price_thb))
    overview=re.sub('\r\n', '', e.select_one('div.model_overview_list_txt').text.strip())
    records.append((model, price_thb, overview, img_url))
df=pd.DataFrame(records, columns=['model', 'price_thb', 'overview', 'img_url'])
df

,model,price_thb,overview,img_url
0,Honda City,579500,เปลี่ยนทุกมาตรฐานของรถซิตี้คาร์ให้พร้อมขับเคลื่อนพาคุณไปไกลเกินกว่าทุกความคาดหมาย,https://www.honda.co.th/uploads/car_models/8477b370d221d06284c8390fc856056299.png
1,Honda City e:HEV,839000,สัมผัสขุมพลังความแรง ประหยัดน้ำมันดีเยี่ยม พร้อมเทคโนโลยีความปลอดภัยอัจฉริยะ ให้คุณพุ่งทะยานสู่อนาคตได้อย่างมั่นใจ,https://www.honda.co.th/uploads/car_models/adefeba31088626dfb607d41ea6229cd50.png
2,Honda City Hatchback,599000,ซิตี้คาร์แฮทช์แบ็ก พร้อมพื้นที่อเนกประสงค์ด้านหลัง และเครื่องยนต์ 1.0 ลิตร เทอร์โบ ให้คุณมูฟไปใช้ชีวิตในแบบตัวเอง,https://www.honda.co.th/uploads/car_models/eea370d2de1e067a56e5e2325a2d9b6463.png
3,Honda Jazz,555000,รถแฮทช์แบ็ก 5 ประตู สำหรับคนรุ่นใหม่ที่มีไลฟ์สไตล์หลากหลาย ให้คุณใช้ชีวิตได้อย่างไร้ขีดจำกัด,https://www.honda.co.th/uploads/car_models/15660956dd6d61ee6bdae8ca873367ac47.png
4,Honda Civic,874000,ที่มาพร้อมเทคโนโลยีความปลอดภัยล้ำสมัย Honda SENSING ให้ความมั่นใจในทุกเส้นทาง,https://www.honda.co.th/uploads/car_models/c365708102fada98707247cc5705b07c82.png
5,Honda Civic Hatchback,1229000,รถแฮทช์แบ็ก 5 ประตู ดีไซน์สปอร์ต โฉบเฉี่ยว ขับขี่สนุกเร้าใจไปกับเครื่องยนต์ VTEC TURBO,https://www.honda.co.th/uploads/car_models/98a14ee47ffb6b0b6f18bd2a2596108b18.png
6,Honda Accord,1475000,ยนตรกรรมสปอร์ตพรีเมียม นิยามใหม่แห่งเทคโนโลยีการขับเคลื่อน สู่โลกบทใหม่แห่งยนตรกรรม,https://www.honda.co.th/uploads/car_models/350749aa28f4cf3d0189aea07ab2a8e779.png
7,Honda CR-V,1369000,เอสยูวีระดับพรีเมียม ดีไซน์หรูหรา แข็งแกร่งทุกมิติ พร้อมพลังขับเคลื่อนด้วยเครื่องยนต์ดีเซลและเบนซิน,https://www.honda.co.th/uploads/car_models/52c345d514984a2edbd78e2a4728de2871.png
8,Honda HR-V,949000,เอสยูวีระดับคอมแพคท์ สะดวกครบครัน พร้อมตอบโจทย์การใช้งานที่หลากหลาย เข้ากับไลฟ์สไตล์คนรุ่นใหม่,https://www.honda.co.th/uploads/car_models/bc924e17caac7397b49ec508ad94555876.png
9,Honda BR-V,765000,รถแอคทีฟสปอร์ตครอสโอเวอร์ ที่พร้อมพาคุณออกไปลุยทุกเส้นทางอันท้าทาย สู่ทุกจุดหมายในทุกสไตล์ที่เป็นคุณ,https://www.honda.co.th/uploads/car_models/d94a659445fd8bf46370c573fca14ddc50.png


In [17]:
def to_html_img(url, alt=None, width=100):
    if alt==None:
        alt=''
    return f'<img alt="{alt}" src="{url}" width="{width}"/>'

def to_html_a(url, text):
    if text==None:
        text=''
    return f'<a href="{url}">{text}</a>'

def save_img(content, imgfilename):
    '''
    save image to file
    '''
    with open(imgfilename, 'wb') as f:
        f.write(content)
        
        
def download_img(url, imgfilename=None):
    '''
    download image from url
    '''
    r=requests.get(url)
    if imgfilename==None:
        imgfilename = f'{str(uuid.uuid1())[:8]}.png'
    save_img(r.content, imgfilename)
    print(f'{imgfilename} saved')

In [22]:
df['img']=df.apply(lambda r: to_html_img(r['img_url'], r['model'], 250), axis=1)
df[:5]

,model,price_thb,overview,img_url,img
0,Honda City,579500,เปลี่ยนทุกมาตรฐานของรถซิตี้คาร์ให้พร้อมขับเคลื่อนพาคุณไปไกลเกินกว่าทุกความคาดหมาย,https://www.honda.co.th/uploads/car_models/8477b370d221d06284c8390fc856056299.png,"<img alt=""Honda City"" src=""https://www.honda.co.th/uploads/car_models/8477b370d221d06284c8390fc856056299.png"" width=""250""/>"
1,Honda City e:HEV,839000,สัมผัสขุมพลังความแรง ประหยัดน้ำมันดีเยี่ยม พร้อมเทคโนโลยีความปลอดภัยอัจฉริยะ ให้คุณพุ่งทะยานสู่อนาคตได้อย่างมั่นใจ,https://www.honda.co.th/uploads/car_models/adefeba31088626dfb607d41ea6229cd50.png,"<img alt=""Honda City e:HEV"" src=""https://www.honda.co.th/uploads/car_models/adefeba31088626dfb607d41ea6229cd50.png"" width=""250""/>"
2,Honda City Hatchback,599000,ซิตี้คาร์แฮทช์แบ็ก พร้อมพื้นที่อเนกประสงค์ด้านหลัง และเครื่องยนต์ 1.0 ลิตร เทอร์โบ ให้คุณมูฟไปใช้ชีวิตในแบบตัวเอง,https://www.honda.co.th/uploads/car_models/eea370d2de1e067a56e5e2325a2d9b6463.png,"<img alt=""Honda City Hatchback"" src=""https://www.honda.co.th/uploads/car_models/eea370d2de1e067a56e5e2325a2d9b6463.png"" width=""250""/>"
3,Honda Jazz,555000,รถแฮทช์แบ็ก 5 ประตู สำหรับคนรุ่นใหม่ที่มีไลฟ์สไตล์หลากหลาย ให้คุณใช้ชีวิตได้อย่างไร้ขีดจำกัด,https://www.honda.co.th/uploads/car_models/15660956dd6d61ee6bdae8ca873367ac47.png,"<img alt=""Honda Jazz"" src=""https://www.honda.co.th/uploads/car_models/15660956dd6d61ee6bdae8ca873367ac47.png"" width=""250""/>"
4,Honda Civic,874000,ที่มาพร้อมเทคโนโลยีความปลอดภัยล้ำสมัย Honda SENSING ให้ความมั่นใจในทุกเส้นทาง,https://www.honda.co.th/uploads/car_models/c365708102fada98707247cc5705b07c82.png,"<img alt=""Honda Civic"" src=""https://www.honda.co.th/uploads/car_models/c365708102fada98707247cc5705b07c82.png"" width=""250""/>"


In [23]:
HTML(df[:][['model', 'price_thb', 'overview', 'img']].to_html(escape=False))

,model,price_thb,overview,img
0,Honda City,579500,เปลี่ยนทุกมาตรฐานของรถซิตี้คาร์ให้พร้อมขับเคลื่อนพาคุณไปไกลเกินกว่าทุกความคาดหมาย,
1,Honda City e:HEV,839000,สัมผัสขุมพลังความแรง ประหยัดน้ำมันดีเยี่ยม พร้อมเทคโนโลยีความปลอดภัยอัจฉริยะ ให้คุณพุ่งทะยานสู่อนาคตได้อย่างมั่นใจ,
2,Honda City Hatchback,599000,ซิตี้คาร์แฮทช์แบ็ก พร้อมพื้นที่อเนกประสงค์ด้านหลัง และเครื่องยนต์ 1.0 ลิตร เทอร์โบ ให้คุณมูฟไปใช้ชีวิตในแบบตัวเอง,
3,Honda Jazz,555000,รถแฮทช์แบ็ก 5 ประตู สำหรับคนรุ่นใหม่ที่มีไลฟ์สไตล์หลากหลาย ให้คุณใช้ชีวิตได้อย่างไร้ขีดจำกัด,
4,Honda Civic,874000,ที่มาพร้อมเทคโนโลยีความปลอดภัยล้ำสมัย Honda SENSING ให้ความมั่นใจในทุกเส้นทาง,
5,Honda Civic Hatchback,1229000,รถแฮทช์แบ็ก 5 ประตู ดีไซน์สปอร์ต โฉบเฉี่ยว ขับขี่สนุกเร้าใจไปกับเครื่องยนต์ VTEC TURBO,
6,Honda Accord,1475000,ยนตรกรรมสปอร์ตพรีเมียม นิยามใหม่แห่งเทคโนโลยีการขับเคลื่อน สู่โลกบทใหม่แห่งยนตรกรรม,
7,Honda CR-V,1369000,เอสยูวีระดับพรีเมียม ดีไซน์หรูหรา แข็งแกร่งทุกมิติ พร้อมพลังขับเคลื่อนด้วยเครื่องยนต์ดีเซลและเบนซิน,
8,Honda HR-V,949000,เอสยูวีระดับคอมแพคท์ สะดวกครบครัน พร้อมตอบโจทย์การใช้งานที่หลากหลาย เข้ากับไลฟ์สไตล์คนรุ่นใหม่,
9,Honda BR-V,765000,รถแอคทีฟสปอร์ตครอสโอเวอร์ ที่พร้อมพาคุณออกไปลุยทุกเส้นทางอันท้าทาย สู่ทุกจุดหมายในทุกสไตล์ที่เป็นคุณ,


# Save img to files

In [26]:
df[:].apply(lambda r: download_img(r['img_url'], f"{r['model'].replace(':', '_')}.jpg"), axis=1)

Honda City.jpg saved
Honda City e_HEV.jpg saved
Honda City Hatchback.jpg saved
Honda Jazz.jpg saved
Honda Civic.jpg saved
Honda Civic Hatchback.jpg saved
Honda Accord.jpg saved
Honda CR-V.jpg saved
Honda HR-V.jpg saved
Honda BR-V.jpg saved
Honda Mobilio.jpg saved
Honda Brio Amaze.jpg saved
Honda Brio.jpg saved


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
dtype: object

In [27]:
df[['model', 'price_thb', 'overview', 'img_url']].to_csv('honda.csv', index=False)